In [17]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os, time
from IPython.display import display, Markdown

In [18]:
options = webdriver.ChromeOptions()
options.add_argument(' — ignore-certificate-errors')
options.add_argument(' — incognito')
options.add_argument(' — headless')

In [19]:
driver = webdriver.Chrome('./driver/chromedriver', options=options)
dir_path = os.getcwd()
url = f'file://{dir_path}/index.html'
driver.get(url)

/var/folders/_t/gs7cg9h922s7rt1lrsw8d50w0000gn/T/ipykernel_68825/1858395511.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver', options=options)


In [20]:
countries_with_colors = { 'australia': [2,3,4],'india':[2,4,6],'us':[2,4,6]}
algorithms = ['backtracking','forward_checking']
heuristics = ['mrv','dh','noh']


In [21]:
mapping_ids = {
    'number_of_colors': 'numColors',
    'countries': 'map-option',
    'algorithms': 'search-option',
    'heuristics': 'forward_checking-options-container'
}   
numeber_of_tries = 5

In [22]:
def choose_color(num_of_colors):
    input_element = driver.find_element(By.ID,mapping_ids['number_of_colors'])    
    input_element.clear()
    input_element.send_keys(num_of_colors)

def choose_country(value):
    dropdown_element = driver.find_element(By.ID,mapping_ids['countries'])    
    dropdown_element = Select(dropdown_element)
    dropdown_element.select_by_value(value)

def choose_algorithm(value):
    dropdown_element  = driver.find_element(By.ID,mapping_ids['algorithms'])    
    dropdown_element = Select(dropdown_element)
    dropdown_element.select_by_value(value)

def choose_heuristic(value):
    div_container  = driver.find_element(By.ID,mapping_ids['heuristics'])    
    radio_button = div_container.find_element(By.ID,value)
    radio_button.click()

In [23]:
def run_csp():
    button_element = driver.find_element(By.ID,'runCSP')
    button_element.click()

def csp_executing():
    status = driver.find_element(By.ID,'status')
    if status.text.strip() =='':
        return True
    return False

def get_execution_time():
    execution_time = driver.find_element(By.ID,'timer')
    return float(execution_time.text.replace('Execution time: ','').replace(' ms',''))

def csp_solved_status():
    status = driver.find_element(By.ID,'status')
    if status.text == 'Solution Status: Found':
        return 'found'
    return 'not_found'

In [24]:
def run_backtracking():
    results = []
    for i in range(numeber_of_tries):
        for country in countries_with_colors:
            for color in countries_with_colors[country]: 
                result = {}
                result['num_of_colors'] = color
                result['country'] = country
                result['algorithm'] = 'backtracking'
                choose_color(color)
                choose_country(country)
                choose_algorithm('backtracking')
                run_csp()
                while csp_executing():
                    time.sleep(2)
                result['execution_time'] = get_execution_time()
                result['execution_id'] = i+1
                result['solved_status'] = csp_solved_status()
                results.append(result)
                driver.refresh()
                time.sleep(1)
    return results

In [25]:
def run_forward_checking():
    results = []
    for i in range(numeber_of_tries):
        for country in countries_with_colors:
            for color in countries_with_colors[country]:
                for heuristic in heuristics: 
                    result = {}
                    result['num_of_colors'] = color
                    result['country'] = country
                    choose_color(color)
                    choose_country(country)
                    choose_algorithm('forward_checking')
                    choose_heuristic(heuristic)
                    if heuristic =='noh':
                        result['algorithm'] = 'forward_checking_without_heuristic'
                    else:
                        result['algorithm'] = f'forward_checking_{heuristic}'
                    run_csp()
                    while csp_executing():
                        time.sleep(2)
                    result['execution_time'] = get_execution_time()
                    result['execution_id'] = i+1
                    result['solved_status'] = csp_solved_status()
                    results.append(result)
                    driver.refresh()
                    time.sleep(1)
    return results

In [26]:
backtracking_results = run_backtracking()

In [27]:
forward_checking_results = run_forward_checking()

In [28]:
df = pd.DataFrame([item for sublist in [backtracking_results,forward_checking_results] for item in sublist])

In [29]:
def show_agg(df,country):
    df = df[df['country']==country]
    display(Markdown(f'## Country: {country.upper()}, Map Coloring Average Execution Time and Number of Executions'))
    grouped_df = df.groupby(['country','num_of_colors','algorithm','solved_status'])
    result = grouped_df.agg({'execution_time': 'mean', 'execution_id': 'count'}).rename(columns={'execution_time': 'avg execution time (ms)', 'execution_id': 'number of executions'})
    return result

In [30]:
show_agg(df,'australia')

## Country: AUSTRALIA, Map Coloring Average Execution Time and Number of Executions

avg execution time (ms)  \
country   num_of_colors algorithm                          solved_status                            
australia 2             backtracking                       not_found                       106.94   
                        forward_checking_dh                not_found                        94.44   
                        forward_checking_mrv               not_found                        87.88   
                        forward_checking_without_heuristic not_found                        97.06   
          3             backtracking                       found                           129.28   
                        forward_checking_dh                found                            81.12   
                        forward_checking_mrv               found                            81.62   
                        forward_checking_without_heuristic found                            86.40   
          4             backtracking                       found                           137.02   
                        forward_checking_dh                found                            84.96   
                        forward_checking_mrv               found                            74.62   
                        forward_checking_without_heuristic found                            79.66   

                                                                          number of executions  
country   num_of_colors algorithm                          solved_status                        
australia 2             backtracking                       not_found                         5  
                        forward_checking_dh                not_found                         5  
                        forward_checking_mrv               not_found                         5  
                        forward_checking_without_heuristic not_found                         5  
          3             backtracking                       found                             5  
                        forward_checking_dh                found                             5  
                        forward_checking_mrv               found                             5  
                        forward_checking_without_heuristic found                             5  
          4             backtracking                       found                             5  
                        forward_checking_dh                found                             5  
                        forward_checking_mrv               found                             5  
                        forward_checking_without_heuristic found                             5

In [31]:
show_agg(df,'india')

## Country: INDIA, Map Coloring Average Execution Time and Number of Executions

avg execution time (ms)  \
country num_of_colors algorithm                          solved_status                            
india   2             backtracking                       not_found                      1700.44   
                      forward_checking_dh                not_found                      1526.64   
                      forward_checking_mrv               not_found                      1334.72   
                      forward_checking_without_heuristic not_found                      1932.46   
        4             backtracking                       found                          1576.24   
                      forward_checking_dh                found                          1390.08   
                      forward_checking_mrv               found                          1718.36   
                      forward_checking_without_heuristic found                          1422.82   
        6             backtracking                       found                          1757.50   
                      forward_checking_dh                found                          1591.60   
                      forward_checking_mrv               found                          1496.62   
                      forward_checking_without_heuristic found                          1533.94   

                                                                        number of executions  
country num_of_colors algorithm                          solved_status                        
india   2             backtracking                       not_found                         5  
                      forward_checking_dh                not_found                         5  
                      forward_checking_mrv               not_found                         5  
                      forward_checking_without_heuristic not_found                         5  
        4             backtracking                       found                             5  
                      forward_checking_dh                found                             5  
                      forward_checking_mrv               found                             5  
                      forward_checking_without_heuristic found                             5  
        6             backtracking                       found                             5  
                      forward_checking_dh                found                             5  
                      forward_checking_mrv               found                             5  
                      forward_checking_without_heuristic found                             5

In [32]:
show_agg(df,'us')

## Country: US, Map Coloring Average Execution Time and Number of Executions

avg execution time (ms)  \
country num_of_colors algorithm                          solved_status                            
us      2             backtracking                       not_found                       122.94   
                      forward_checking_dh                not_found                       127.52   
                      forward_checking_mrv               not_found                       148.36   
                      forward_checking_without_heuristic not_found                       136.42   
        4             backtracking                       found                           818.02   
                      forward_checking_dh                found                           120.88   
                      forward_checking_mrv               found                           125.32   
                      forward_checking_without_heuristic found                           590.88   
        6             backtracking                       found                           103.38   
                      forward_checking_dh                found                           106.14   
                      forward_checking_mrv               found                           113.46   
                      forward_checking_without_heuristic found                           110.68   

                                                                        number of executions  
country num_of_colors algorithm                          solved_status                        
us      2             backtracking                       not_found                         5  
                      forward_checking_dh                not_found                         5  
                      forward_checking_mrv               not_found                         5  
                      forward_checking_without_heuristic not_found                         5  
        4             backtracking                       found                             5  
                      forward_checking_dh                found                             5  
                      forward_checking_mrv               found                             5  
                      forward_checking_without_heuristic found                             5  
        6             backtracking                       found                             5  
                      forward_checking_dh                found                             5  
                      forward_checking_mrv               found                             5  
                      forward_checking_without_heuristic found                             5